# Lab 2: Damped Oscillator

First, we install packages that are used in this notebook, but are not
installed in Google Colab by default.

If you are not running this notebook in Google Colab (e.g., if you are
running it locally) you can skip the first code block, provided that you
have all necessary packages installed in the environment this notebook
is running in.

If you are running this notebook in Google Colab and any of the required
packages are absent from the environment you are executing this notebook
in, then you will need to install those packages in the environment by
modifying the following code block.

In [ ]:
%pip install uncertainties

Second, we load the Python packages needed for this notebook.

In [ ]:
from __future__ import annotations  # noqa: F404

from io import StringIO

import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import pandas as pd
import scipy.optimize
from IPython.display import Markdown
from matplotlib.axes import Axes
from matplotlib.figure import Figure
from uncertainties import ufloat

Now, we can begin following the lab instructions (reproduced in this
notebook within block quotes, with some minor alterations).

> You have a damped oscillator set up, that is also coupled to the
> driver.

## Part 1: Free Oscillations

> 1. First, investigate the free oscillations. Determine the angular
>    frequency $\omega$ and the damping term $\gamma$ in the equation
>
>    $$
>      x\left(t\right) = A_0 e^{-\frac{\gamma t}{2}}
>      \cos\left(\omega t + \alpha\right)
>    $$
>
>    based on your measurement of the free oscillations with damping.
>    See [amplitude.pdf] for instructions about how to measure
>    $A\left(t\right) = A_0 e^{-\gamma t / 2}$, the time dependent
>    amplitude.

**Note:** This notebook allows you to fit your measurement data with the
function for the displacement of a damped harmonic oscillator, rather
than following the procedure in [amplitude.pdf].

[amplitude.pdf]: http://solidstate.physics.sunysb.edu/teaching/2025/phy300/lab/amplitude.pdf

In the following code block, we define the functions:

- `dho_func()`: the displacement of a damped harmonic oscillator as a
  function of time,
- `dho_amp()`: the decaying amplitude of the oscillator as a function of
  time, and
- `dho_wave()`: the normalized, dimensionless oscillations as a function
  of time.

You will later fit your measured data for the free oscillations with
`dho_func()`.  The other functions, `dho_amp()` and `dho_wave()` are
used to define `dho_func()`.

In [ ]:
def dho_func(
    time: npt.NDArray,
    amplitude_init: float,
    angular_frequency: float,
    damping_term: float,
    phase: float,
    offset: float,
) -> npt.NDArray:
    R"""
    The damped oscillation function.

    Parameters
    ----------
    time : numpy.ndarray
        An array of time values at which the damped oscillation is to be
        evaluated.
    amplitude_init : float
        The initial amplitude at time zero.
    angular_frequency : float
        The angular frequency of the oscillations.
    damping_term : float
        The damping term, which "is the reciprocal of time required for
        the energy to decrease to :math:`1 / e` of its initial value"
        [1]_.
    phase : float
        The phase of the oscillation at time zero.
    offset : float
        The scalar offset of the oscillations, i.e., the value about
        which the oscillations are centered.

    Returns
    -------
    displacement : numpy.ndarray
        The displacement of the damped oscillator at each time specified
        by the parameter `time`.

    Notes
    -----
    The equation for the displacement, :math:`x`, as a function of time,
    :math:`t`, is

    .. math::

        x\left(t\right) = A_0 e^{-\cfrac{\gamma t}{2}}
        \cos\left(\omega t + \alpha) + x_0 \, ,

    where :math:`A_0` is the initial amplitude, :math:`\gamma` is the
    damping term, :math:`\omega` is the angular frequency,
    :math:`\alpha` is the phase, and :math:`x_0` is the offset.

    References
    ----------
    .. [1] A. P. French, *Vibrations and Waves*, The M.I.T. Introductory
       Physics Series (W. W. Norton & Company, Inc., New York, 1971), p.
       67.
    """
    return (
        dho_amp(time, amplitude_init, damping_term)
        * dho_wave(time, angular_frequency, phase)
        + offset
    )


def dho_amp(
    time: npt.NDArray,
    amplitude_init: float,
    damping_term: float,
) -> npt.NDArray:
    R"""
    The amplitude component of the damped oscillator function.

    Parameters
    ----------
    time : numpy.ndarray
        An array of time values at which the amplitude is to be
        evaluated.
    amplitude_init : float
        The initial amplitude at time zero.
    damping_term : float
        The damping term, which "is the reciprocal of time required for
        the energy to decrease to :math:`1 / e` of its initial value"
        [1]_.

    Returns
    -------
    amplitude : numpy.ndarray
        An array of the amplitudes component evaluated at each time
        specified in the array `time`.  Note that the "amplitude
        component" be multiplied by the wave component (:func:`dho_wave`
        function) to obtain the damped oscillation.

    Notes
    -----
    The formula for the amplitude, :math:`A`, as a function of time,
    :math:`t`, is

    .. math::

        A\left(t\right) = A_0 e^{-\cfrac{\gamma t}{2}} \, ,

    where :math:`A_0` is the initial amplitude and :math:`\gamma` is the
    damping term.

    References
    ----------
    .. [1] A. P. French, *Vibrations and Waves*, The M.I.T. Introductory
       Physics Series (W. W. Norton & Company, Inc., New York, 1971), p.
       67.
    """
    return amplitude_init * np.exp(-damping_term * time / 2)


def dho_wave(
    time: npt.NDArray,
    angular_frequency: float,
    phase: float,
) -> npt.NDArray:
    R"""
    The wave component of the damped oscillator function.

    Parameters
    ----------
    time : numpy.ndarray
        An array of time values at which the wave is to be evaluated.
    angular_frequency : float
        The angular frequency of the oscillations.
    phase : float
        The oscillation phase at time zero.

    Returns
    -------
    wave : numpy.ndarray
        An array of the wave amplitudes evaluated at each time specified
        in the array `time`.  Note that the "wave amplitude" is
        dimensionless and must be multiplied by the :func:`dho_amp`
        function to obtain the amplitude of the damped oscillation.

    Notes
    -----
    The formula for the wave component, :math:`u`, as a function of
    time, :math:`t`, is

    .. math::

        u\left(t\right) = \cos\left(\omega t + \alpha) \, ,

    where :math:`\omega` is the angular frequency and :math:`\alpha` is
    the phase.
    """
    return np.cos(angular_frequency * time + phase)

Copy your data from the text file and paste it into the `raw_data`
variable in the following code block.  The content you paste must be in
a multiline string, which begins and ends with three quote characters
(by convention, double quotes are used, but single quotes will also
work, so long as you do not single and double quotes).

In [ ]:
raw_data = """Vernier Format 2
FILENAME DATE TIME
Run NUMBER
Time	Position	Velocity	Acceleration
T	P	V	A
s	m	m/s	m/s^2

0.00	0.284	-0.227	0.413
...
30.00	0.275	0.004	-0.087
"""

In the next block, your measurement data is read from `raw_data`.

In [ ]:
# Get first line of ``raw_data``.
first_line = next(iter(raw_data.splitlines()))
# Check ``raw-data`` begins with the Vernier header and set the value of
# ``header`` accordingly (so that ``pandas`` will know to skip it).
if first_line == "Vernier Format 2":
    header = 5
else:
    header = None
del first_line
# Read in the data.
df = pd.read_csv(
    StringIO(raw_data),
    sep="\t",
    header=header,
    names=("t", "x", "v", "a"),
    index_col=False,
    engine="python",
)
del header
# Display the data (note that it will be truncated to avoid taking up an
# excessive amount of space).
display(df)

Check that when your data is displayed by the previous code block, the
`t` column must begin with zero end end with the time you set the
measurement to run for.
**If this is not the case, something went wrong.**

If the first value in the `t` column is not zero, then its possible that
``raw_data`` only included a partial header.  If this is the case, you
can probably fix the issue by changing the line
```python3
    header=header,
```
(in the code block above) to
```python3
    header=NUMBER_OF_LINES_MINUS_ONE,
```
where `NUMBER_OF_LINES_MINUS_ONE` is one less than the number of header
lines present in ``raw_data``, and then running the code block above
again.

If the procedure in the previous paragraph is not applicable or does not
work, then you should contact the TAs for assistance.  When contacting
the TAs for assistance, please include your modified copy of this
notebook.

The following code block will plot your measurement data for the
position/displacement of the oscillator as a function of time.  **Make
sure that the units indicated in the plot match those of your data.**

In [ ]:
fig, ax = plt.subplots()
fig.tight_layout()
ax.set_title(R"Position vs. Time Measurements")
ax.set_xlabel(R"Time, $t$ $\left[\text{s}\right]$")
ax.set_ylabel(R"Position, $x$ $\left[\text{m}\right]$")
ax.plot(df.t, df.x, ".-");

Next, we will use the plot above to estimate the parameters of our fit,
which we will enter into the `p0` tuple in the next code block.  The
elements of `p0` are your guesses for:

1. the initial amplitude (`amplitude_init`),
2. the angular frequency (`angular_frequency`),
3. the damping factor (`daming_factor`),
4. the phase (`phase`), and
5. the offset (`offset`).

You don't need to get the exact values for `p0` as it is just an initial
guess.  In fact, there's a good chance that the example values will be
sufficiently close for the curve fitting to be successful, so you may
want to try running the next code block without altering it to see if it
works.

In [ ]:
p0: tuple[float, float, float, float, float] = (
    0.01, # amplitude_init
    10.0, # angular_frequency
    0.01, # damping_term
    0.0, # phase
    0.0, # offset
)
popt: npt.NDArray
pcov: npt.NDArray
infodict: dict
mesg: str
ier: int
popt, pcov, infodict, mesg, ier = scipy.optimize.curve_fit(
    dho_func,
    df.t,
    df.x,
    p0,
    full_output=True,
)

display(Markdown("**Curve Fit Message:**"))
print(mesg)

perr = np.sqrt(np.diag(pcov))

display(
    Markdown(
        f"""**Condition Number of Covariance Matrix:** {
            np.linalg.cond(pcov):.2e}"""
    )
)

The output of the `scipy.optimize.curve_fit()` function in the code
block above are:

- `popt`: an array of optimized fit parameters,
- `pcov`: the covariance matrix of the fit parameters,
- `infodict`: a dictionary of information about the fit,
- `mesg`: a status message, and
- `ier`: an integer indicating the convergence status.

The variable `perr` is an array of the uncertainties of the optimized
fit parameters calculated from `pcov`.

In the output of the code block above, the "Curve Fit Message" is simply
the `mesg` variable and the "Condition Number of Covariance Matrix" is
the result of passing `pcov` to the `numpy.linalg.cond()` function.  A
large value for the condition number (i.e., of order $10^{N}$ where $N
\gg 1$) indicates issues with the fit function.

You can find details regarding the output of the `scipy.optimize.curve_fit()`
function variables as well as `perr` and the condition number in [the
documentation for the `scipy.optimize.curve_fit()` function](curve_fit).
See also [the documentation for the `numpy.linalg.cond()`
function](cond) for details regarding the calculation of the condition
number of a matrix.

[cond]: https://numpy.org/doc/2.1/reference/generated/numpy.linalg.cond.html
[curve_fit]: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

In the next code block, we use the `ufloat()` function from the
`uncertainties` package to store the optimized fit parameters and their
uncertainties as the variables:

- `amplitude_init`,
- `angular_frequency`,
- `damping_term`,
- `phase`, and
- `offset`.

We then use these variables when we display the values of each variable
of the fit function.  By default, `uncertainties` package uses the
[Particle Data Group](PDG) rounding rules to determine the number of
significant digits to display for uncertainties (see also [the user
guide for the `uncertainties` package](uncertainties-user_guide) for
details).

Finally, plot the measured data, fit function, and fit residuals.

[PDG]: https://pdg.lbl.gov/2010/reviews/rpp2010-rev-rpp-intro.pdf
[uncertainties-user_guide]: https://pythonhosted.org/uncertainties/user_guide.html

In [ ]:
amplitude_init = ufloat(popt[0], perr[0])
angular_frequency = ufloat(popt[1], perr[1])
damping_term = ufloat(popt[2], perr[2])
phase = ufloat(popt[3], perr[3])
offset = ufloat(popt[4], perr[4])
display(Markdown(fR"""
Fit function:

$$
    x\left(t\right)
    = A_0 e^{{-\gamma t / 2}} \cos\left(\omega t + \varphi\right) + x_0
$$

Amplitude (initial): $$A_0 = {amplitude_init:L}\,\text{{m}}$$
Angular frequency: $$\omega = {angular_frequency:L}\,\text{{rad}}/\text{{s}}$$
Damping term: $$\gamma = {damping_term:L}\,\text{{rad}}/\text{{s}}$$
Phase: $$\varphi = {phase:L}\,\text{{rad}}$$
Offset: $$x_0 = {offset:L}\,\text{{m}}$$"""))

t_curve = np.linspace(np.min(df.t), np.max(df.t), (df.t.count() - 1) * 10 + 1)
df["res"] = df.x - dho_func(df.t, *popt)  # type: ignore

fig: Figure
axs: tuple[Axes, Axes]
fig, axs = plt.subplots(2, 1, sharex=True)
fig.tight_layout()
axs[-1].set_xlabel(R"Time, $t$ $\left[\text{s}\right]$")
axs[0].set_ylabel(R"Displacement, $x$ $\left[\text{m}\right]$")
axs[0].scatter(df.t, df.x, s=1, color="C0", label=R"measured", zorder=2.1)
axs[0].plot(t_curve, dho_func(t_curve, *popt), color="k", label=R"fit", linewidth=0.5)
axs[0].legend()
axs[1].set_ylabel(R"Residual $\left[\text{m}\right]$")
axs[1].scatter(df.t, df.res, s=1, color="r", label=R"residual")
axs[1].axhline(0, color="k", linewidth=0.5, zorder=0.9)
axs[1].legend();

> 2. Calculate the quality factor $Q$.

The quality factor is defined by $Q = \cfrac{\omega_0}{\gamma}$.

**Note:** The angular frequency (`angular_frequency`) of the damped free
oscillations, $\omega$, obtained by fitting your data is different from
the angular frequency of the (undamped) simple harmonic oscillator,
$\omega_0$.  We from here on, we will denote the natural angular
frequency of free oscillations of the damped harmonic oscillator as
$\omega_1$, rather than as $\omega$, to avoid confusion.

We know that $\omega_1$ is related to $\omega_0$ by the equation

$$ \omega_1 = \sqrt{\omega_0^2 - \cfrac{\gamma^2}{4}} \, . $$

In the limit where $\omega_1 \gg \gamma$, we find that

$$ Q \approx \cfrac{\omega_1}{\gamma} \, . $$

**Calculating the quality factor is &ldquo;left as an exercise to the
reader,&rdquo;** i.e., the code to do so is not provided in this
notebook.  Also, you **don't** have to write code to calculate $Q$, you
can simply do it with a calculator.  **Don't forget to calculate the
uncertainty of $Q$, $\sigma_Q$, and to show your formula for $\sigma_Q$
in your submission.**